In [3]:
# Import necessary packages
import pandas as pd
import numpy  as np

# Reading Data
timedf = pd.read_csv('PrepSiteTimeMatrix.csv', nrows=41, dtype={0:int}, header=0, index_col=0)

print(timedf)

        174   1109   1102    405    218    461    464    214    455    278  \
174    0.00  14.74  14.70  22.88  17.73  20.03  10.45  15.25  16.43  18.49   
1109  15.59   0.00  24.24  31.45  27.26  29.87  22.13  27.66  25.96  29.81   
1102  15.32  24.64   0.00  10.79   5.00   8.25  10.60  13.25   2.29  11.01   
405   22.45  30.26  11.43   0.00  10.86   9.18  16.07  10.61  12.81   6.68   
218   17.63  26.95   4.89  10.73   0.00   8.19  11.63  13.45   5.22  10.95   
461   19.01  29.60   8.32   8.76   8.64   0.00  10.22   6.80  10.05   5.99   
464   10.20  21.20   9.86  16.01  11.76  10.70   0.00   8.49  11.00  11.18   
214   15.03  26.77  13.08  10.82  13.40   8.48   8.32   0.00  14.81   4.88   
455   16.26  25.58   2.42  12.34   5.33   9.80  11.38  14.80   0.00  12.56   
278   18.54  28.91  11.52   6.51  11.13   6.96  10.95   4.75  13.08   0.00   
437   19.90  30.56  10.75   6.26  10.18   5.74  11.12   6.39  12.13   2.51   
413   14.50  25.51   8.44  11.56   8.73   6.47   5.72   6.78  10

In [5]:
# This is python notebook inside of pycharm
print('helo')

helo


In [6]:
from ortools import pywraplp

ImportError: cannot import name 'pywraplp' from 'ortools' (/Users/aarij/.local/lib/python3.7/site-packages/ortools/__init__.py)